In [1]:
# Importing Libraries:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
import findspark
import pyspark.sql.functions as F

from pyspark.sql.functions import concat_ws, col
findspark.init()

In [2]:
# Reading the data
data = pd.read_csv('Renaissance_Hotel.csv', encoding= 'unicode_escape') 
data=data.drop(['Unnamed: 0'], axis=1)
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,duration_of_stay
0,Renaissance Hotel,0,13,2018,July,27,1,0,1,1,...,A,0,No Deposit,304.0,0,Transient,75.00,0,0,1
1,Renaissance Hotel,0,14,2018,July,27,1,0,2,2,...,A,0,No Deposit,240.0,0,Transient,98.00,0,1,2
2,Renaissance Hotel,0,9,2018,July,27,1,0,2,2,...,C,0,No Deposit,303.0,0,Transient,103.00,0,1,2
3,Renaissance Hotel,1,85,2018,July,27,1,0,3,2,...,A,0,No Deposit,240.0,0,Transient,82.00,0,1,3
4,Renaissance Hotel,1,75,2018,July,27,1,0,3,2,...,D,0,No Deposit,15.0,0,Transient,105.50,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27166,Renaissance Hotel,0,212,2017,August,35,31,2,8,2,...,A,1,No Deposit,143.0,0,Transient,89.75,0,0,10
27167,Renaissance Hotel,0,169,2017,August,35,30,2,9,2,...,E,0,No Deposit,250.0,0,Transient-Party,202.27,0,1,11
27168,Renaissance Hotel,0,204,2017,August,35,29,4,10,2,...,E,0,No Deposit,250.0,0,Transient,153.57,0,3,14
27169,Renaissance Hotel,0,211,2017,August,35,31,4,10,2,...,D,0,No Deposit,40.0,0,Contract,112.80,0,1,14


In [3]:
# Number of rows and columns in the data set
rows,cols=data.shape

In [4]:
# Names of all the column
for col in data.columns:
    print(col)

hotel
is_canceled
lead_time
arrival_date_year
arrival_date_month
arrival_date_week_number
arrival_date_day_of_month
stays_in_weekend_nights
stays_in_week_nights
adults
children
babies
meal
country
distribution_channel
is_repeated_guest
previous_cancellations
previous_bookings_not_canceled
reserved_room_type
assigned_room_type
booking_changes
deposit_type
agent
days_in_waiting_list
customer_type
adr
required_car_parking_spaces
total_of_special_requests
duration_of_stay


In [5]:
#Total number of duplicated rows--> no duplicated row 
data.duplicated().sum()

1

In [6]:
# Dropping duplicated rows
data=data.drop_duplicates()

In [7]:
# Checking if there are missing data
data.isnull().values.any()

False

In [8]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/06/09 16:59:15 WARN Utils: Your hostname, Lojins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
23/06/09 16:59:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/09 16:59:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Loading the data into spark dataframe
df = spark.createDataFrame(data) 
# Checking that columns were inserted correctly 
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: long (nullable = true)
 |-- lead_time: long (nullable = true)
 |-- arrival_date_year: long (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: long (nullable = true)
 |-- arrival_date_day_of_month: long (nullable = true)
 |-- stays_in_weekend_nights: long (nullable = true)
 |-- stays_in_week_nights: long (nullable = true)
 |-- adults: long (nullable = true)
 |-- children: double (nullable = true)
 |-- babies: long (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: long (nullable = true)
 |-- previous_cancellations: long (nullable = true)
 |-- previous_bookings_not_canceled: long (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullable = true)
 |-- booking_changes: long (nullable = true)
 |-- deposit_type: string 

Promotion One

In [92]:
#Room A is the most room type that is cancelled alot 
query55=df.filter(F.col('is_canceled') == 1)
query55=query55.groupBy('reserved_room_type', 'is_canceled')\
    .count().\
    show()

+------------------+-----------+-----+
|reserved_room_type|is_canceled|count|
+------------------+-----------+-----+
|                 C|          1|  265|
|                 A|          1| 3363|
|                 H|          1|  237|
|                 G|          1|  598|
|                 D|          1| 1470|
|                 F|          1|  151|
|                 E|          1| 1096|
+------------------+-----------+-----+



In [93]:
#After calculating the duration of stay and we grouped by room type(A) we find out that 
from pyspark.sql.functions import col

query_df = df.filter( (col('is_canceled') == 1)& (col('reserved_room_type') == "A"))
query_df=query_df.groupBy("reserved_room_type","duration_of_stay").count()
query_df=query_df.sort("count",ascending=False)
query_df.show()

+------------------+----------------+-----+
|reserved_room_type|duration_of_stay|count|
+------------------+----------------+-----+
|                 A|               7|  694|
|                 A|               2|  481|
|                 A|               3|  480|
|                 A|               4|  439|
|                 A|               1|  329|
|                 A|               5|  324|
|                 A|               6|  212|
|                 A|               8|   93|
|                 A|              10|   80|
|                 A|              14|   73|
|                 A|               9|   65|
|                 A|              11|   23|
|                 A|              12|   16|
|                 A|              13|    8|
|                 A|              18|    8|
|                 A|              21|    6|
|                 A|              25|    5|
|                 A|               0|    5|
|                 A|              15|    4|
|                 A|            

After computing this query we find out that the highest number of cancellation is during the reservation of days from  2-7 days

Pomotion will be added for 20% if a guest reserved room type A starting from 3 days reservations 

Promotion Two

In [94]:
# We assume that families are at least one children or one baby with at least one adult
query_F = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') >= 1) | (col('babies') >= 1)))
print("The count of family who did cancel their reservation",query_F.count())

#  Getting adults who did  cancel their reservations
query_A = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') == 0) & (col('babies') == 0)))
print("The count of adults who did cancel their reservation",query_A.count())

The count of family who did cancel their reservation 1311
The count of adults who did cancel their reservation 5869


Based on this result we will investigate more to see why Adults cancel reservations more 

In [95]:
#  Getting adults who did  cancel their reservations
query_A = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') == 0) & (col('babies') == 0)))
print("The count of adults who did cancel their reservation",query_A.count())

# Going to work on Adults who has history of cancellations:
query_A1=query_A.filter((col('previous_bookings_not_canceled') > 0))

# Checking from where do these Adults cancel reseervation
query_A1= query_A1.groupby('country')\
.agg(count("*").alias("count"))\
.sort("count", ascending=False)
query_A1.show()

The count of adults who did cancel their reservation 5869
+-------+-----+
|country|count|
+-------+-----+
|    PRT|    4|
|    IRL|    2|
|    ESP|    1|
+-------+-----+



Clearly, Adults from Portugal cancel a lot reservations. 

We are going to investigate more why the adults from Portugal cancel reservations more than others.

In [96]:
#  Getting adults who did  cancel their reservations
query_A = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') == 0) & (col('babies') == 0)))
print("The count of adults who did cancel their reservation",query_A.count())

# Going to work on Adults who has history of cancellations:
query_A1=query_A.filter((col('previous_bookings_not_canceled') > 0))
query_A1=query_A1.filter((col('country') == 'PRT'))

# Checking the customer type of these Adults who cancel reseervation
query_A1= query_A1.groupby('customer_type')\
.agg(count("*").alias("count"))\
.sort("count", ascending=False)
query_A1.show()

The count of adults who did cancel their reservation 5869
+-------------+-----+
|customer_type|count|
+-------------+-----+
|    Transient|    4|
+-------------+-----+



Based on the results of this query, it is suggested to offer the following:

1) 5% for all adults 
2) extra 5% for adults coming from Portugal 
3) extra 5% for transient adults

Promotion Three

In [97]:
query_df3 = df.filter((col('is_canceled') == 1))
query_df3=query_df3.groupBy('previous_cancellations', 'is_repeated_guest')\
    .count()
query_df3=query_df3.sort("previous_cancellations")
query_df3.show() 
# Show the query results


+----------------------+-----------------+-----+
|previous_cancellations|is_repeated_guest|count|
+----------------------+-----------------+-----+
|                     0|                0| 6930|
|                     0|                1|    3|
|                     1|                0|  203|
|                     1|                1|   22|
|                     2|                0|   12|
|                     2|                1|    1|
|                     3|                0|    4|
|                    14|                0|    1|
|                    24|                0|    1|
|                    25|                0|    2|
|                    26|                0|    1|
+----------------------+-----------------+-----+



As we filter the query to have all the people who cancelled there reservations , we find out that people that aren't repeated guests are the most guests that cancelled the reservation.
So to enhance the number of customers' reservations we will give this offer to the people who didn't cancell before (previous_cancellations=0) and they are repeated guest . 

The promotion will be :**two** days extra  with no fees applied 

In this Query, my target is to increase the number of regular customers , so we can assure that we will gain customer attraction

Promotion Four

In [98]:
#Here i find out that the ones who reserved alotwithout any cancellation aren't put in the waiting list 
query77=df.filter(col('is_canceled') == 0)
query77=query77.groupBy('is_canceled', 'days_in_waiting_list')\
    .count()
query77=query77.sort("days_in_waiting_list")
query77.show()

+-----------+--------------------+-----+
|is_canceled|days_in_waiting_list|count|
+-----------+--------------------+-----+
|          0|                   0|19900|
|          0|                   1|    2|
|          0|                   2|    1|
|          0|                   4|    1|
|          0|                   5|    1|
|          0|                   6|    4|
|          0|                  11|    1|
|          0|                  14|    4|
|          0|                  18|    1|
|          0|                  22|    1|
|          0|                  34|    2|
|          0|                  37|    1|
|          0|                  43|    2|
|          0|                  44|    1|
|          0|                  50|    1|
|          0|                  52|    1|
|          0|                  60|    4|
|          0|                  61|    2|
|          0|                  64|    3|
|          0|                  65|    4|
+-----------+--------------------+-----+
only showing top

From the previous Query we will apply promotion for the guests in the waiting list so it will be : a complimentary upgrade to a higher category room . This can include better views, more space, enhanced amenities, or additional features like a private balcony or jacuzzi.